### Projeto 3: Tuning

#### Etapa 1: Importação das bibliotecas

In [1]:
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetBinaryClassifier #pytorch nao tem validacao cruzada

#### Etapa 2: Base de dados

In [2]:
np.random.seed(123)
torch.manual_seed(123) # Mesmos pesos

In [3]:
previsores = pd.read_csv('Arquivos/entradas_breast.csv')
classes = pd.read_csv('Arquivos/saidas_breast.csv')

In [4]:
classes.shape

(569, 1)

In [5]:
previsores = np.array(previsores, dtype = 'float32')
classes = np.array(classes, dtype='float32').squeeze(1)

In [6]:
classes.shape

(569,)

#### Etapa 3: Classe para estrutura da rede neural

In [7]:
class classificador_torch(nn.Module):
    def __init__(self, activation, neurons, initializer):
        super().__init__()

        #30 -> 16 -> 16 -> 1
        self.dense0 = nn.Linear(30, neurons)
        initializer(self.dense0.weight)
        self.activation0 = activation
        self.dense1 = nn.Linear(neurons, neurons)
        initializer(self.dense1.weight)
        self.activation1 = activation
        self.dense2 = nn.Linear(neurons, 1)
        initializer(self.dense2.weight)

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        X = self.dense1(X)
        X = self.activation1(X)
        X = self.dense2(X)

        return X


#### Etapa 4: Skorch 

In [8]:
classificador_sklearn = NeuralNetBinaryClassifier(module=classificador_torch, 
                                                  lr = 0.001, 
                                                  optimizer__weight_decay = 0.0001,
                                                  train_split=False) # Train split é falso porque faremos a validação cruzada
                                                  

#### Etapa 5: Tuning

In [9]:
params = {'batch_size': [10], 
'max_epochs': [100],
'optimizer': [torch.optim.Adam, torch.optim.SGD],
'criterion': [torch.nn.BCEWithLogitsLoss, torch.nn.HingeEmbeddingLoss],
'module__activation': [F.relu, torch.tanh],
'module__neurons': [8, 16],
'module__initializer': [torch.nn.init.uniform_, torch.nn.init.normal_]}

In [10]:
params

{'batch_size': [10],
 'max_epochs': [100],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD],
 'criterion': [torch.nn.modules.loss.BCEWithLogitsLoss,
  torch.nn.modules.loss.HingeEmbeddingLoss],
 'module__activation': [<function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  <function torch._VariableFunctionsClass.tanh>],
 'module__neurons': [8, 16],
 'module__initializer': [<function torch.nn.init.uniform_(tensor: torch.Tensor, a: float = 0.0, b: float = 1.0) -> torch.Tensor>,
  <function torch.nn.init.normal_(tensor: torch.Tensor, mean: float = 0.0, std: float = 1.0) -> torch.Tensor>]}

In [11]:
grid_search = GridSearchCV(estimator = classificador_sklearn, param_grid=params, scoring = 'accuracy', cv = 2)
grid_search = grid_search.fit(previsores, classes)

  epoch    train_loss     dur
-------  ------------  ------
      1    27299.8738  0.0450
      2    24343.2038  0.0350
      3    21607.9801  0.0320
      4    19135.1209  0.0330
      5    16931.4981  0.0320
      6    14986.2359  0.0370
      7    13263.7468  0.0370
      8    11731.9746  0.0370
      9    10367.9507  0.0340
     10     9150.4845  0.0320
     11     8060.7275  0.0340
     12     7080.4200  0.0340
     13     6192.2921  0.0330
     14     5382.8836  0.0330
     15     4640.1406  0.0350
     16     3952.5612  0.0360
     17     3309.9056  0.0390
     18     2702.8676  0.0330
     19     2122.0807  0.0360
     20     1558.6194  0.0340
     21     1003.4049  0.0340
     22      478.4398  0.0350
     23      138.4234  0.0340
     24       90.7292  0.0330
     25       88.3919  0.0340
     26       77.7052  0.0340
     27       70.4368  0.0342
     28       64.3089  0.0350
     29       58.6982  0.0340
     30       54.5392  0.0350
     31       52.0839  0.0360
     32   

c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_scorer.py", line 219, in __call__
    return self._score(
  File "c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_scorer.py", line 261, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_scorer.py", line 71, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\U

      6    11274.6513  0.0390
      7     7975.8562  0.0390
      8     4782.3244  0.0380
      9     1560.5291  0.0360
     10     -819.3868  0.0370
     11    -2348.5971  0.0370
     12    -4007.4772  0.0390
     13    -5880.3792  0.0380
     14    -8021.4942  0.0360
     15   -10492.1325  0.0380
     16   -13360.6949  0.0380
     17   -16683.9709  0.0420
     18   -20506.0638  0.0390
     19   -24901.3789  0.0360
     20   -29989.3188  0.0370
     21   -35877.8542  0.0370
     22   -42698.5741  0.0340
     23   -50580.1572  0.0350
     24   -59628.4122  0.0400
     25   -69939.8693  0.0380
     26   -81608.1081  0.0390
     27   -94723.7103  0.0390
     28  -109377.9478  0.0400
     29  -125664.7087  0.0370
     30  -143687.4379  0.0390
     31  -163567.8968  0.0360
     32  -185456.1298  0.0360
     33  -209542.3115  0.0380
     34  -236058.6903  0.0370
     35  -265263.6985  0.0380
     36  -297405.4857  0.0370
     37  -332684.0772  0.0390
     38  -371237.4748  0.0370
     39  -

c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [0.72596368 0.62215221 0.83833086 0.6239066  0.74867803 0.62916976
 0.76102051 0.62916976 0.62741537 0.62741537 0.62741537 0.62741537
 0.68007784 0.60974178 0.64839387 0.66603657        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]
  warnings.warn(


      3    56722.2148  0.0680
      4    43101.2074  0.0690
      5    32402.7327  0.0690
      6    23948.5464  0.0750
      7    17173.4956  0.0730
      8    11619.9703  0.0710
      9     7049.6777  0.0720
     10     3203.4680  0.0670
     11      440.1589  0.0700
     12      162.2341  0.0700
     13      156.4794  0.0710
     14      145.9702  0.0670
     15      139.3940  0.0680
     16      128.6295  0.0690
     17      119.0552  0.0710
     18      113.2228  0.0680
     19      108.2722  0.0770
     20      100.7649  0.0730
     21      100.1233  0.0670
     22       92.0195  0.0690
     23       87.7824  0.0720
     24       82.0457  0.0690
     25       78.8340  0.0680
     26       76.5950  0.0710
     27       74.4955  0.0700
     28       71.2428  0.0740
     29       66.6542  0.0680
     30       64.5609  0.0670
     31       61.3889  0.0720
     32       63.6442  0.0700
     33       67.7277  0.0670
     34       64.0120  0.0710
     35       60.0192  0.0720
     36   

In [12]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_
melhores_parametros, melhor_precisao

({'batch_size': 10,
  'criterion': torch.nn.modules.loss.BCEWithLogitsLoss,
  'max_epochs': 100,
  'module__activation': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  'module__initializer': <function torch.nn.init.uniform_(tensor: torch.Tensor, a: float = 0.0, b: float = 1.0) -> torch.Tensor>,
  'module__neurons': 16,
  'optimizer': torch.optim.adam.Adam},
 0.8383308623671856)